# Queueboard data analysis notebook (pull requests)

## Preliminaries

In [ ]:
# Run me to download latest data from GitHub
from download_artifact import download_and_extract_latest_successful_workflow_artifacts
info = download_and_extract_latest_successful_workflow_artifacts(
    repo="leanprover-community/queueboard-core",
    workflow="upload_backup.yaml",
    out_dir="./data",
    artifact_name="analytics-datasets",
    branch="master",
    search_limit=100,  # change this if you expect there to be > 100 failed runs before the first successful one
)
info

In [ ]:
# Load PR and PR event data
from pathlib import Path
import polars as pl
from datetime import timezone

data_dir = Path("data")

# Names of columns that should be converted to datetime
DT_COLS = [
    "created_at",
    "updated_at",
    "gh_created_at",
    "gh_updated_at",
    "last_synced_at",
    "commits_earliest_synced_at",
    # "timeline_earliest_synced_at", # TODO: why does this end up as Float64?
    "engagement_synced_at",
    "occurred_at",
    "closed_at",
    "merged_at",
]

def parse_dt_cols(df_raw):
  return df_raw.with_columns([
    pl.col(c)
      # strangely, the %:::z format doesn't seem to work, but %#z seems to, even both are mentioned in the chrono crate docs
      # https://docs.rs/chrono/latest/chrono/format/strftime/index.html
      # (mentioned in polars docs https://docs.pola.rs/api/python/dev/reference/expressions/api/polars.Expr.dt.strftime.html)
      .str.strptime(pl.Datetime('us', timezone.utc), format="%Y-%m-%d %T%.f%#z", strict=False)
      .alias(c)
    for c in DT_COLS
    if c in df_raw.columns
  ])

df_prs_raw = pl.read_parquet(data_dir / "syncer_pullrequest.parquet")
df_prs = parse_dt_cols(df_prs_raw)

df_events_raw = pl.read_parquet(data_dir / "syncer_prtimelineevent.parquet")
df_events = parse_dt_cols(df_events_raw)

## Open intervals

In [ ]:
import polars as pl
from datetime import datetime, timezone

def build_pr_open_intervals(
    df_prs: pl.DataFrame,
    df_events: pl.DataFrame,
    *,
    asof: pl.Expr | None = None,
) -> pl.DataFrame:
    """
    Build PR open intervals from PR created_at + CLOSED/REOPENED events.

    Returns columns:
      - pull_request_id
      - start (Datetime)
      - end (Datetime | null)
      - is_open_ended (bool)
      - end_effective (Datetime)
      - duration (Duration)
      - duration_hours (float)
      - duration_days (float)
    """
    if asof is None:
        asof = pl.lit(datetime.now(tz=timezone.utc))

    # --- PR creation change (+1) ---
    prs_changes = (
        df_prs
        .select([
            pl.col("id").alias("pull_request_id"),
            pl.col("gh_created_at").alias("occurred_at"),
        ])
        .with_columns([
            pl.lit("CREATED").alias("etype"),
            pl.lit(1, dtype=pl.Int32).alias("change"),
            pl.lit(0, dtype=pl.Int32).alias("order"),
        ])
        .drop_nulls(["occurred_at"])
    )

    # --- Event changes ---
    ev_changes = (
        df_events
        .filter(pl.col("type").is_in(["CLOSED", "REOPENED"]))
        .select([
            pl.col("pull_request_id"),
            pl.col("occurred_at"),
            pl.col("type").alias("etype"),
        ])
        .with_columns([
            pl.when(pl.col("etype") == "CLOSED").then(-1).otherwise(1).cast(pl.Int32).alias("change"),
            pl.when(pl.col("etype") == "REOPENED").then(1).otherwise(2).cast(pl.Int32).alias("order"),
        ])
        .drop_nulls(["occurred_at"])
    )

    changes = (
        pl.concat([prs_changes, ev_changes], how="vertical")
        .sort(["pull_request_id", "occurred_at", "order"])
        .with_columns([
            pl.col("change").cum_sum().over("pull_request_id").alias("open_count"),
        ])
        .with_columns([
            pl.col("open_count").shift(1).over("pull_request_id").fill_null(0).alias("prev_open_count"),
        ])
        .with_columns([
            ((pl.col("prev_open_count") <= 0) & (pl.col("open_count") > 0)).alias("start_flag"),
            ((pl.col("prev_open_count") > 0) & (pl.col("open_count") <= 0)).alias("end_flag"),
        ])
        .with_columns([
            pl.col("start_flag").cast(pl.Int32).cum_sum().over("pull_request_id").alias("start_idx"),
            pl.col("end_flag").cast(pl.Int32).cum_sum().over("pull_request_id").alias("end_idx"),
        ])
    )

    starts = (
        changes
        .filter(pl.col("start_flag"))
        .select([
            pl.col("pull_request_id"),
            pl.col("start_idx").alias("interval_idx"),
            pl.col("occurred_at").alias("start"),
        ])
    )

    ends = (
        changes
        .filter(pl.col("end_flag"))
        .select([
            pl.col("pull_request_id"),
            pl.col("end_idx").alias("interval_idx"),
            pl.col("occurred_at").alias("end"),
        ])
    )

    intervals = (
        starts
        .join(ends, on=["pull_request_id", "interval_idx"], how="left")
        .select(["pull_request_id", "start", "end"])
        .sort(["pull_request_id", "start"])
        .with_columns([
            pl.col("end").is_null().alias("is_open_ended"),
            pl.coalesce([pl.col("end"), asof]).alias("end_effective"),
        ])
        .with_columns([
            (pl.col("end_effective") - pl.col("start")).alias("duration"),
        ])
        .with_columns([
            (pl.col("duration").dt.total_seconds() / 3600.0).alias("duration_hours"),
            (pl.col("duration").dt.total_seconds() / 86400.0).alias("duration_days"),
        ])
    )

    return intervals


In [ ]:
df_intervals = build_pr_open_intervals(df_prs, df_events)

# Or fix as-of time for reproducibility:
# df_intervals_pl = build_pr_open_intervals(
#     df_prs, df_events,
#     asof=pl.lit(pl.datetime(2025, 12, 17, 0, 0, 0, time_unit="us"))
# )

## Open PRs vs time

In [ ]:
import polars as pl
from datetime import datetime, timezone

def open_prs_per_day(
    intervals: pl.DataFrame,
    *,
    start_col: str = "start",
    end_effective_col: str = "end_effective",
    asof: datetime | None = None,   # optional cutoff
) -> pl.DataFrame:
    """
    Compute number of open PRs per day.

    Counts a PR as open on a day if it is open at any point during that day.
    For each interval [start, end_effective], we:
      +1 on start_date
      -1 on (end_date + 1 day)

    Returns DataFrame with columns:
      - date (Date)
      - delta (i64)
      - open_prs (i64)
    """

    if asof is None:
        asof = datetime.now(tz=timezone.utc)

    asof_lit = pl.lit(asof)

    df = (
        intervals
        .with_columns([
            pl.col(start_col),
            pl.col(end_effective_col),
        ])
        # cap end_effective at asof (optional but usually what you want)
        .with_columns([
            pl.when(pl.col(end_effective_col) > asof_lit)
              .then(asof_lit)
              .otherwise(pl.col(end_effective_col))
              .alias(end_effective_col)
        ])
        .with_columns([
            pl.col(start_col).dt.date().alias("start_date"),
            pl.col(end_effective_col).dt.date().alias("end_date"),
        ])
    )

    # +1 deltas on start_date
    starts = (
        df.group_by("start_date")
          .agg(pl.len().cast(pl.Int64).alias("delta"))
          .rename({"start_date": "date"})
    )

    # -1 deltas on day after end_date
    ends = (
        df.with_columns((pl.col("end_date") + pl.duration(days=1)).alias("date"))
          .group_by("date")
          .agg((-pl.len().cast(pl.Int64)).alias("delta"))
    )

    deltas = (
        pl.concat([starts, ends], how="vertical")
          .group_by("date")
          .agg(pl.sum("delta").alias("delta"))
          .sort("date")
    )

    # Build a complete daily index and fill missing deltas with 0
    min_date = df.select(pl.min("start_date")).item()
    max_date = df.select(pl.max("end_date")).item()
    all_days = pl.DataFrame({
        "date": pl.date_range(min_date, max_date, interval="1d", eager=True)
    })

    out = (
        all_days
        .join(deltas, on="date", how="left")
        .with_columns(pl.col("delta").fill_null(0))
        .sort("date")
        .with_columns(pl.col("delta").cum_sum().alias("open_prs"))
    )

    return out

In [ ]:
import altair
daily = open_prs_per_day(df_intervals)
daily.plot.line(x="date", y="open_prs")

## Interval plot

In [ ]:
import polars as pl
from datetime import datetime, timezone
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.dates as mdates
import numpy as np


def prepare_swimlane_polars(
    intervals: pl.DataFrame,
    *,
    asof: datetime | None = None,
    max_prs: int | None = None,   # optional downsample: keep earliest max_prs PRs
) -> pl.DataFrame:
    if asof is None:
        asof = datetime.now(tz=timezone.utc)

    # ensure end_effective exists
    df = intervals.with_columns(
        pl.coalesce([pl.col("end"), pl.lit(asof)]).alias("end_effective")
    )

    # first open per PR -> defines order
    order = (
        df.group_by("pull_request_id")
          .agg(pl.min("start").alias("first_open"))
          .sort("first_open")
          .with_row_index("y")              # y = 0..n-1 in sorted order
    )

    if max_prs is not None:
        order = order.head(max_prs)

    # attach y back to each interval
    out = (
        df.join(order.select(["pull_request_id", "y"]), on="pull_request_id", how="inner")
          .select(["pull_request_id", "start", "end_effective", "y"])
          .sort(["y", "start"])
    )
    return out

def plot_swimlane_matplotlib(swim: pl.DataFrame, *, figsize=(12, 12), linewidth=0.5, title=None):
    start = swim["start"].to_numpy()
    end   = swim["end_effective"].to_numpy()
    y     = swim["y"].to_numpy()

    x0 = mdates.date2num(start)
    x1 = mdates.date2num(end)

    segments = [((a, yi), (b, yi)) for a, b, yi in zip(x0, x1, y)]

    fig, ax = plt.subplots(figsize=figsize)
    lc = LineCollection(segments, linewidths=linewidth)
    lc.set_rasterized(True)
    ax.add_collection(lc)

    ax.set_xlim(x0.min(), x1.max())
    ax.set_ylim(-1, y.max() + 1)
    ax.xaxis_date()
    fig.autofmt_xdate()

    ax.set_xlabel("Time")
    ax.set_ylabel("PR (earliest at bottom)")
    ax.set_title(title or "PR Open Intervals (swimlane)")
    plt.tight_layout()
    plt.show()


In [ ]:
swim_all = prepare_swimlane_polars(df_intervals)   # all PRs
plot_swimlane_matplotlib(swim_all, figsize=(5, 25), linewidth=0.4)

## Filtering Classes of Intervals

In [ ]:
import polars as pl

def enrich_intervals_with_prs(
    intervals: pl.DataFrame,
    prs: pl.DataFrame,
    *,
    pr_id_col: str = "pull_request_id",
    prs_id_col: str = "id",
    pr_cols: list[str] | None = None,
) -> pl.DataFrame:
    """
    Left-join PR metadata onto interval rows.

    intervals: must have pull_request_id
    prs: must have id and any columns you want to join
    pr_cols: PR columns to bring over (excluding id). If None, uses a sensible default.
    """
    if pr_cols is None:
        pr_cols = [
            "number", "state", "is_draft",
            "repository_id", "author_id",
            "base_ref_name", "head_ref_name",
            "title",
            "additions", "deletions", "changed_files_count",
            "gh_created_at", "closed_at", "merged_at",
        ]

    cols_present = [c for c in pr_cols if c in prs.columns]
    prs_small = prs.select([pl.col(prs_id_col).alias(pr_id_col), *[pl.col(c) for c in cols_present]])

    return intervals.join(prs_small, on=pr_id_col, how="left")


In [ ]:
df_intervals_enriched = enrich_intervals_with_prs(df_intervals, df_prs)

In [ ]:
import re

# title related filters

def expr_title_contains(substr: str, *, case_insensitive: bool = True) -> pl.Expr:
    # literal substring match (fast, safe)
    return pl.col("title").fill_null("").str.to_lowercase().str.contains(
        substr.lower(), literal=True
    ) if case_insensitive else pl.col("title").fill_null("").str.contains(substr, literal=True)

def expr_title_any(substrs: list[str], *, case_insensitive: bool = True) -> pl.Expr:
    e = pl.lit(False)
    for s in substrs:
        e = e | expr_title_contains(s, case_insensitive=case_insensitive)
    return e

def expr_title_regex(pattern: str, *, case_insensitive: bool = True) -> pl.Expr:
    # regex match
    if case_insensitive:
        # prefix with (?i) for case-insensitive
        pattern = "(?i)" + pattern
    return pl.col("title").fill_null("").str.contains(pattern, literal=False)

def expr_title_exclude_any(substrs: list[str], *, case_insensitive: bool = True) -> pl.Expr:
    return ~expr_title_any(substrs, case_insensitive=case_insensitive)

# PR property filters

def expr_repo_in(repo_ids: list[int]) -> pl.Expr:
    return pl.col("repository_id").is_in(repo_ids)

def expr_author_in(author_ids: list[int]) -> pl.Expr:
    return pl.col("author_id").is_in(author_ids)

def expr_base_branch_in(branches: list[str]) -> pl.Expr:
    return pl.col("base_ref_name").is_in(branches)

def expr_state_is(state: str) -> pl.Expr:
    return pl.col("state") == state

def expr_is_draft(is_draft: bool = True) -> pl.Expr:
    return pl.col("is_draft") == is_draft

# size / churn filters

def expr_additions_between(lo: int | None = None, hi: int | None = None) -> pl.Expr:
    e = pl.lit(True)
    if lo is not None:
        e = e & (pl.col("additions").fill_null(0) >= lo)
    if hi is not None:
        e = e & (pl.col("additions").fill_null(0) <= hi)
    return e

def expr_churn_between(lo: int | None = None, hi: int | None = None) -> pl.Expr:
    churn = pl.col("additions").fill_null(0) + pl.col("deletions").fill_null(0)
    e = pl.lit(True)
    if lo is not None:
        e = e & (churn >= lo)
    if hi is not None:
        e = e & (churn <= hi)
    return e

# interval / date filters

from datetime import datetime, timezone
from typing import Union
import pandas as pd

DateLike = Union[str, datetime]

def _to_utc_datetime(x: DateLike | None) -> datetime | None:
    if x is None:
        return None

    if isinstance(x, datetime):
        # ensure tz-aware UTC
        if x.tzinfo is None:
            return x.replace(tzinfo=timezone.utc)
        return x.astimezone(timezone.utc)

    if isinstance(x, str):
        try:
            dt = pd.to_datetime(x, utc=True)
        except Exception as e:
            raise ValueError(f"Could not parse datetime string: {x!r}") from e
        return dt.to_pydatetime()

    raise TypeError(f"Expected datetime or str, got {type(x)}")

def expr_interval_started_between(
    *,
    start_after: DateLike | None = None,
    start_before: DateLike | None = None,
) -> pl.Expr:
    start_after_dt = _to_utc_datetime(start_after)
    start_before_dt = _to_utc_datetime(start_before)

    e = pl.lit(True)

    if start_after_dt is not None:
        e = e & (pl.col("start") >= pl.lit(start_after_dt))

    if start_before_dt is not None:
        e = e & (pl.col("start") < pl.lit(start_before_dt))

    return e

def expr_only_closed(only_closed: bool = True) -> pl.Expr:
    return pl.col("is_open_ended") == (not only_closed)

# wrapper for filters

def filter_intervals(df: pl.DataFrame, *exprs: pl.Expr) -> pl.DataFrame:
    """
    Apply multiple Polars boolean expressions as filters.
    """
    if not exprs:
        return df
    e = pl.lit(True)
    for ex in exprs:
        e = e & ex
    return df.filter(e)

In [ ]:
df_closed = filter_intervals(df_intervals_enriched, expr_only_closed(True))
print('df_closed', len(df_closed))

df_open = filter_intervals(df_intervals_enriched, expr_only_closed(False))
print('df_open', len(df_open))
df_open_feat = filter_intervals(df_open, expr_title_regex(r"^feat"))
print('df_open_feat', len(df_open_feat))

# doesn't count PRs that actually got merged
df_merged = filter_intervals(df_closed, expr_title_regex(r"^\[Merged by Bors\] -"))
print('df_merged', len(df_merged))
df_merged_feat = filter_intervals(df_closed, expr_title_regex(r"^\[Merged by Bors\] -\s+[fF]eat"))
print('df_merged_feat', len(df_merged_feat))
df_merged_nonfeat = filter_intervals(df_closed, expr_title_regex(r"^\[Merged by Bors\] -\s+[^\sfF]"))
print('df_merged_nonfeat', len(df_merged_nonfeat))

In [ ]:
df_merged_2021 = filter_intervals(df_merged, expr_interval_started_between(start_after='2021-01-01', start_before='2022-01-01'))
print('df_merged_2021', len(df_merged_2021))
df_merged_2022 = filter_intervals(df_merged, expr_interval_started_between(start_after='2022-01-01', start_before='2023-01-01'))
print('df_merged_2022', len(df_merged_2022))
df_merged_2023 = filter_intervals(df_merged, expr_interval_started_between(start_after='2023-01-01', start_before='2024-01-01'))
print('df_merged_2023', len(df_merged_2023))
df_merged_2024 = filter_intervals(df_merged, expr_interval_started_between(start_after='2024-01-01', start_before='2025-01-01'))
print('df_merged_2024', len(df_merged_2024))
df_merged_2025 = filter_intervals(df_merged, expr_interval_started_between(start_after='2025-01-01', start_before='2026-01-01'))
print('df_merged_2025', len(df_merged_2025))

In [ ]:
df_merged_feat_after_port = filter_intervals(df_merged_feat, expr_interval_started_between(start_after='2023-08-01'))
print('df_merged_feat_after_port', len(df_merged_feat_after_port))
df_merged_nonfeat_after_port = filter_intervals(df_merged_nonfeat, expr_interval_started_between(start_after='2023-08-01'))
print('df_merged_nonfeat_after_port', len(df_merged_nonfeat_after_port))

## Open Durations

### Helper functions

In [ ]:
import numpy as np
import polars as pl

def get_x(df: pl.DataFrame, col: str) -> np.ndarray:
    x = df.select(pl.col(col)).to_numpy().ravel().astype(float)
    x = x[np.isfinite(x) & (x > 0)]
    return x


In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import lognorm, weibull_min, fisk

def plot_duration_hist(
    df: pl.DataFrame,
    *,
    col="duration_days",
    bins=100,
    logx=False,
    logy=False,
    exponential_fit=False,
    title=None,
):
    x = get_x(df, col)
    if x.size < 2:
        print("Not enough data to plot.")
        return

    # bins
    if logx:
        lo = max(x.min(), np.nextafter(0, 1))
        hi = x.max()
        bin_edges = np.logspace(np.log10(lo), np.log10(hi), bins + 1)
    else:
        bin_edges = np.linspace(x.min(), x.max(), bins + 1)

    fig, ax = plt.subplots(figsize=(10, 4))
    counts, edges, _ = ax.hist(x, bins=bin_edges)

    if logx:
        ax.set_xscale("log")
    if logy:
        ax.set_yscale("log")

    if exponential_fit and logy:
        N = x.size
        lam = 1.0 / x.mean()
        centers = 0.5 * (edges[:-1] + edges[1:])
        widths = np.diff(edges)
        expected = N * lam * np.exp(-lam * centers) * widths
        ax.plot(centers, expected, linewidth=3, label=f"Exponential (λ={lam:.3g})")
        ax.legend()

    ax.set_xlabel(col.replace("_", " "))
    ax.set_ylabel("Count")
    ax.set_title(title or f"Histogram of {col}")
    plt.tight_layout()
    plt.show()

def plot_lognormal_fit_counts_logbins(df, *, col="duration_days", bins=100, title=None):
    x = get_x(df, col)
    N = x.size
    if N < 2:
        print("Not enough data.")
        return None

    lo = max(x.min(), np.nextafter(0, 1))
    hi = x.max()
    edges = np.logspace(np.log10(lo), np.log10(hi), bins + 1)
    centers = np.sqrt(edges[:-1] * edges[1:])

    sigma, loc, scale = lognorm.fit(x, floc=0)
    mu = np.log(scale)

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.hist(x, bins=edges, density=False, alpha=0.6)

    cdf = lognorm.cdf(edges, s=sigma, loc=loc, scale=scale)
    expected = N * np.diff(cdf)
    ax.plot(centers, expected, linewidth=3, label=f"Lognormal (μ={mu:.2f}, σ={sigma:.2f})")

    ax.set_xscale("log")
    ax.set_xlabel(col.replace("_", " "))
    ax.set_ylabel("Count per log bin")
    ax.set_title(title or "Lognormal fit (counts)")
    ax.legend()
    plt.tight_layout()
    plt.show()

    return {"mu": float(mu), "sigma": float(sigma), "n": int(N)}

def plot_weibull_fit_counts_logbins(df, *, col="duration_days", bins=100, title=None):
    x = get_x(df, col)
    N = x.size
    if N < 2:
        print("Not enough data.")
        return None

    lo = max(x.min(), np.nextafter(0, 1))
    hi = x.max()
    edges = np.logspace(np.log10(lo), np.log10(hi), bins + 1)
    centers = np.sqrt(edges[:-1] * edges[1:])

    k, loc, scale = weibull_min.fit(x, floc=0)

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.hist(x, bins=edges, density=False, alpha=0.6)

    cdf = weibull_min.cdf(edges, k, loc=loc, scale=scale)
    expected = N * np.diff(cdf)
    ax.plot(centers, expected, linewidth=3, label=f"Weibull (k={k:.2f})")

    ax.set_xscale("log")
    ax.set_xlabel(col.replace("_", " "))
    ax.set_ylabel("Count per log bin")
    ax.set_title(title or "Weibull fit (counts)")
    ax.legend()
    plt.tight_layout()
    plt.show()

    return {"k": float(k), "scale": float(scale), "n": int(N)}

def plot_loglogistic_fit_counts_logbins(df, *, col="duration_days", bins=100, title=None):
    x = get_x(df, col)
    N = x.size
    if N < 2:
        print("Not enough data.")
        return None

    lo = max(x.min(), np.nextafter(0, 1))
    hi = x.max()
    edges = np.logspace(np.log10(lo), np.log10(hi), bins + 1)
    centers = np.sqrt(edges[:-1] * edges[1:])

    c, loc, scale = fisk.fit(x, floc=0)

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.hist(x, bins=edges, density=False, alpha=0.6)

    cdf = fisk.cdf(edges, c, loc=loc, scale=scale)
    expected = N * np.diff(cdf)
    ax.plot(centers, expected, linewidth=3, label=f"Log-logistic (c={c:.2f})")

    ax.set_xscale("log")
    ax.set_xlabel(col.replace("_", " "))
    ax.set_ylabel("Count per log bin")
    ax.set_title(title or "Log-logistic fit (counts)")
    ax.legend()
    plt.tight_layout()
    plt.show()

    return {"c": float(c), "scale": float(scale), "n": int(N)}


### Basic plots

In [ ]:
plot_duration_hist(df_closed, logx=True)

In [ ]:
plot_duration_hist(df_open, logx=True)

In [ ]:
plot_duration_hist(df_open, logy=True, exponential_fit=True)

In [ ]:
1/0.00417

In [ ]:
df_open['duration_days'].median()

In [ ]:
plot_duration_hist(df_merged, logx=True)

### Plots with fits

In [ ]:
merged_lognormal_params = plot_lognormal_fit_counts_logbins(df_merged)

In [ ]:
np.exp(merged_lognormal_params['mu'])

In [ ]:
df_merged['duration_days'].median()

In [ ]:
plot_loglogistic_fit_counts_logbins(df_merged)

In [ ]:
plot_weibull_fit_counts_logbins(df_merged)

### More lognormal plots

In [ ]:
plot_lognormal_fit_counts_logbins(df_merged, col="duration_days", bins=100)

In [ ]:
np.exp(0.454)

In [ ]:
df_merged['duration_days'].median()

In [ ]:
plot_lognormal_fit_counts_logbins(df_merged_2021, col="duration_days", bins=100)

In [ ]:
plot_lognormal_fit_counts_logbins(df_merged_2022, col="duration_days", bins=100)

In [ ]:
plot_lognormal_fit_counts_logbins(df_merged_2023, col="duration_days", bins=100)

In [ ]:
plot_lognormal_fit_counts_logbins(df_merged_2024, col="duration_days", bins=100)

In [ ]:
plot_lognormal_fit_counts_logbins(df_merged_2025, col="duration_days", bins=100)

### Combined plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import lognorm

def plot_hist_and_lognormal_fit_overlays(
    dfs: list[pl.DataFrame],
    labels: list[str],
    *,
    col: str = "duration_days",
    bins: int = 100,
    title: str | None = None,
    show_params_in_legend: bool = False,
    hist_linewidth: float = 1.2,
    fit_linewidth: float = 3.0,
):
    if len(dfs) != len(labels):
        raise ValueError("dfs and labels must have the same length.")

    xs = []
    for df in dfs:
        x = get_x(df, col)
        xs.append(x)

    allx = np.concatenate([x for x in xs if x.size])
    if allx.size < 2:
        raise ValueError("Not enough data across datasets to plot.")

    lo = max(allx.min(), np.nextafter(0, 1))
    hi = allx.max()
    edges = np.logspace(np.log10(lo), np.log10(hi), bins + 1)
    centers = np.sqrt(edges[:-1] * edges[1:])

    fig, ax = plt.subplots(figsize=(10, 4))

    for x, label in zip(xs, labels):
        if x.size < 2:
            continue

        # Histogram counts (shared edges)
        hist_counts, _ = np.histogram(x, bins=edges)

        # Step outline histogram
        y_step = np.r_[hist_counts, hist_counts[-1]]
        (hist_line,) = ax.step(edges, y_step, where="post", linewidth=hist_linewidth, alpha=0.9)
        color = hist_line.get_color()

        # Lognormal fit (loc=0) -> expected counts per bin
        sigma, loc, scale = lognorm.fit(x, floc=0)
        mu = np.log(scale)
        cdf = lognorm.cdf(edges, s=sigma, loc=loc, scale=scale)
        expected = x.size * np.diff(cdf)

        fit_label = label
        if show_params_in_legend:
            fit_label = f"{label} (μ={mu:.2f}, σ={sigma:.2f}, n={x.size})"

        ax.plot(centers, expected, color=color, linewidth=fit_linewidth, label=fit_label)

    ax.set_xscale("log")
    ax.set_xlabel(col.replace("_", " "))
    ax.set_ylabel("Count per log bin")
    ax.set_title(title or "Overlaid histograms + lognormal fits")
    ax.legend()
    plt.tight_layout()
    plt.show()


In [ ]:

dfs = [df_merged_2021, df_merged_2022, df_merged_2023, df_merged_2024, df_merged_2025]
labels = ["2021", "2022", "2023", "2024", "2025"]

plot_hist_and_lognormal_fit_overlays(
    dfs,
    labels,
    col="duration_days",
    bins=100,
    title="Merged PR interval durations",
    show_params_in_legend=False,  # set True if you want μ/σ in legend
)

In [ ]:
print(len(df_merged_2021), len(df_merged_2022), len(df_merged_2023), len(df_merged_2024), len(df_merged_2025))

In [ ]:
dfs_feat = [df_merged_feat, df_merged_nonfeat]
labels = ["feat", "non-feat"]

plot_hist_and_lognormal_fit_overlays(
    dfs_feat,
    labels,
    col="duration_days",
    bins=100,
    title="Merged PR interval durations",
    show_params_in_legend=True,
)

In [ ]:
np.exp(1.14)

In [ ]:
np.exp(-0.32)

In [ ]:
dfs_feat_after_port = [df_merged_feat_after_port, df_merged_nonfeat_after_port]
labels = ["feat (after 2023-08-01)", "non-feat (after 2023-08-01)"]

plot_hist_and_lognormal_fit_overlays(
    dfs_feat_after_port,
    labels,
    col="duration_days",
    bins=100,
    title="Merged PR interval durations",
    show_params_in_legend=True,  # set True if you want μ/σ in legend
)

In [ ]:
np.exp(1.57)

In [ ]:
np.exp(-0.22)

## TODO: similar plots for review cycles